In [129]:
import numpy as np
import pandas as pd
import pywt

In [79]:
df = pd.read_csv('Ejemplos/Distribuciones/Binomial-fin.csv',index_col='indice')

In [4]:
def series_periodos(inicio, periodos, freq): 
    serie = pd.date_range(start=inicio, periods=periodos, freq=freq)
    return serie

DESPLAZAMIENTO TEMPORAL Y TRASLACION

In [6]:
def time_shift(series, shift):
    return series.shift(shift)

In [94]:
def traslacion(df,shift):
    df[df.columns[0]] = df[df.columns[0]] + shift
    return df

AGREGAR RUIDO:

In [12]:
def add_noise(series, noise_level=0.1):
    noise = np.random.normal(0, noise_level, len(series))
    return series + noise

In [10]:
def df_ruido(df,noise_level=0.1):
    df[df.columns[0]] = add_noise(df[df.columns[0]],noise_level)
    return df

ESCALADO

In [17]:
def scale(series,factor):
    return series*factor

In [16]:
def df_scale(df,factor):
    df[df.columns[0]] = scale(df[df.columns[0]],factor)
    return df

RECORTE

In [15]:
def crop(series, start, end):
    return series[start:end]

In [20]:
def recorte(df, start,end):
    df[df.columns[0]] = crop(df[df.columns[0]],start,end)
    df=df.dropna()
    return df

In [175]:
def random_cuts(data, n_cuts=5):
    cuts = np.random.randint(0, len(data), size=n_cuts)
    return data[cuts]

In [177]:
def agregar_random_cuts(df,freq,n_cuts=5):
    data = df[df.columns[0]].values
    data_cut = random_cuts(data,n_cuts)
    indice=series_periodos(df.index[0],n_cuts,freq)
    df_cut=pd.DataFrame(data=data_cut,index=indice,columns=df.columns)
    return df_cut
    

INTERPOLACION:

In [172]:
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline

In [33]:
def interpolacion(df,kind,num,freq):
    df=df.reset_index()
    indices=df.index.values
    x =indices 
    print(x)
    y = df[df.columns[1]]
    print(y)
    f = interp1d(x, y, kind=kind) #kind='linear' / 'cubic'
    x_new = np.linspace(0, 4, num=num)  # New x values
    y_new = f(x_new)  # Interpolated y values
    indice=series_periodos(df[df.columns[0]][0],num,freq)
    df_int=pd.DataFrame(data=y_new,index=indice)
    return df_int

In [59]:
def interpolate(data):
    interpolated_data = []
    for i in range(len(data) - 1):
        interpolated_data.append(data[i])
        interpolated_data.append((data[i] + data[i + 1]) / 2)  # Punto intermedio
    interpolated_data.append(data[-1])
    return np.array(interpolated_data)

In [68]:
def punto_medio(df,freq):
    data =df[df.columns[0]]
    a=interpolate(data)
    indice=series_periodos(df.index[0],len(a),freq)
    df_pm=pd.DataFrame(data=a,index=indice)
    return df_pm

In [165]:
def spline_interpolation(data, s=1):
    x = np.arange(len(data))
    spline = UnivariateSpline(x, data, s=s)
    return spline(x)

In [164]:
def interpolacion_spline(df,s):
    data = df[df.columns[0]]
    df[df.columns[0]]=spline_interpolation(data,s)
    return df

RANDOM SAMPLING:

In [48]:
# Randomly sampling with replacement
def sampling(df,size,freq):
    data =df[df.columns[0]]
    sampled_data = np.random.choice(data, size=size, replace=True)
    indice=series_periodos(df.index[0],size,freq)
    df_sampling=pd.DataFrame(data=sampled_data,index=indice,columns=df.columns)
    return df_sampling

TRANSFORMACIÓN MATEMÁTICA:

In [51]:
def agregar_log(df):
    data = df[df.columns[0]]
    log_transformed_data = np.log1p(data)
    df[df.columns[0]] = log_transformed_data
    return df 

In [76]:
def agregar_sqrt(df):
    data = df[df.columns[0]]
    df[df.columns[0]] = np.sqrt(data)
    return df 

In [71]:
def agregar_exp(df,factor):
    data = df[df.columns[0]]
    df[df.columns[0]] = np.exp(data/factor)
    return df 

In [169]:
def agregar_sin(df):
    data = df[df.columns[0]]
    df[df.columns[0]] = np.sin(data)
    return df

In [168]:
def agregar_cos(df):
    data = df[df.columns[0]]
    df[df.columns[0]] = np.cos(data)
    return df

In [167]:
def agregar_trig(df):
    data = df[df.columns[0]]
    df[df.columns[0]] = np.cos(data) + np.sin(data)

In [171]:
def agregar_sigmoid(df):
    data = df[df.columns[0]]
    df[df.columns[0]] =1 / (1 + np.exp(-data))
    return df
   

MIXUP

In [194]:
def mixup(data, alpha=0.2):
    lambda_ = np.random.beta(alpha, alpha)
    indices = np.random.permutation(len(data))
    data_mixup = lambda_ * data + (1 - lambda_) * data[indices]
    return data_mixup

In [193]:
def agregar_mixup(df,alpha=0.2):
    df[df.columns[0]] = mixup(df[df.columns[0]].values,alpha)
    return df

In [195]:
agregar_mixup(df)

,Valor
indice,
2014-01-31,16.165199
2014-02-28,136.623102
2014-03-31,18.777608
2014-04-30,95.252260
2014-05-31,97.385959
...,...
2023-08-31,13.785253
2023-09-30,89.284300
2023-10-31,376.298397


In [124]:
def random_mix(data, n_samples=100):
    mixed_data = []
    for _ in range(n_samples):
        i, j = np.random.choice(len(data), 2, replace=False)
        mixed_data.append((data[i] + data[j]) / 2)
    return np.array(mixed_data)

In [125]:
def agregar_random_mix(df,freq,n_samples):
    sampled_data = random_mix(df[df.columns[0]],n_samples)
    indice=series_periodos(df.index[0],n_samples,freq)
    df_sampling=pd.DataFrame(data=sampled_data,index=indice,columns=df.columns)
    return df_sampling

VENTANAS DESLIZANTES:

In [98]:
def ventanas(df,ventana):
    df[df.columns[0]]=df[df.columns[0]].rolling(window=ventana).mean()
    df = df.dropna()
    return df

PCA:

In [ ]:
from sklearn.decomposition import PCA

data_multidim = np.array([[1.2, 3.4], [5.6, 7.8], [9.1, 2.2]])
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_multidim)
data_pca_jittered = data_pca + np.random.normal(0, 0.1, data_pca.shape)
data_augmented = pca.inverse_transform(data_pca_jittered)

USO DE TECNICAS ESTADÍSTICAS:

In [102]:
def normal(df,freq,size):
    data = df[df.columns[0]]
    mean,std_dev=np.mean(data),np.std(data)
    data_augmented = np.random.normal(mean,std_dev,size=data.shape)
    indice=series_periodos(df.index[0],size,freq)
    df_tf=pd.DataFrame(data=data_augmented,index=indice,columns=df.columns)
    return df_tf

In [108]:
def log_normal(df,freq,sigma,size):
    data = df[df.columns[0]].values
    data_augmented = np.random.lognormal(mean=np.log(data.mean()),sigma=sigma,size=size)
    indice=series_periodos(df.index[0],size,freq)
    df_tf=pd.DataFrame(data=data_augmented,index=indice,columns=df.columns)
    return df_tf

In [127]:
def box_muller_transform(mean, std_dev, size=100):
    u1, u2 = np.random.rand(size), np.random.rand(size)
    z1 = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
    return mean + z1 * std_dev

def box_muller(df,freq,size):
    data = df[df.columns[0]].values
    data_tf=box_muller_transform(data.mean(),data.std(),size)
    indice=series_periodos(df.index[0],size,freq)
    df_tf=pd.DataFrame(data=data_tf,index=indice,columns=df.columns)
    return df_tf


DESPLAZAMIENTO EN BLOQUES:

In [111]:
def despl_bloque(df,block_size):
    data = df[df.columns[0]].values
    shifted_data = []
    for i in range(0,len(data),block_size):
        block = data[i:i+block_size]
        shifted_block = np.roll(block, np.random.randint(1, block_size))
        shifted_data.extend(shifted_block)
    df[df.columns[0]]=np.array(shifted_data)
    return df

ROTACIÓN:

In [116]:
def rotacion(df):
    data = df[df.columns[0]].values
    rotation_steps = np.random.randint(1, len(data))
    df[df.columns[0]]= np.roll(data, rotation_steps)
    return df

RUIDO ARMONICO

In [ ]:
def add_harmonic_noise(df, amplitude=0.1, frequency=0.5):
    data = df[df.columns[0]].values
    time = np.arange(len(data))
    harmonic_noise = amplitude * np.sin(2 * np.pi * frequency * time)
    df[df.columns[0]] = data + harmonic_noise
    return df

BOOTSTRAPPING:

In [ ]:
def agregar_bootstrapping(df,freq):
    # Suponiendo  que df es tu DataFrame con series temporales originales
    synthetic_data = df.sample(frac=1, replace=True).reset_index(drop=True)
    synthetic_data['Valor'] += np.random.normal(0, 0.1, len(synthetic_data))  # Añadir ruido
    indice=series_periodos(df.index[0],len(df)+len(synthetic_data),freq)
    a=pd.concat([df['Valor'],synthetic_data['Valor']])
    a.index=indice
    df_bootstrap=pd.DataFrame(data=a)
    return df_bootstrap

PERTURBACIÓN GAUSSIANA:

In [121]:
def cumulative_gaussian_perturbation(data, std_dev=0.05):
    perturbations = np.random.normal(0, std_dev, len(data))
    cumulative_data = np.cumsum(perturbations) + data
    return cumulative_data

In [118]:
def agregacion_gauss(df,std_dev=0.05):
    df[df.columns[0]] = cumulative_gaussian_perturbation(df[df.columns[0]],std_dev)
    return df

TRANSFORMACIÓN WAVELET:

In [143]:
def wavelet_transform_augmentation(data, wavelet='db1'):
    coeffs = pywt.wavedec(data, wavelet)
    coeffs[0] += np.random.normal(0, 0.1, len(coeffs[0]))  # Añadir ruido al coeficiente principal
    augmented_data = pywt.waverec(coeffs, wavelet)
    return augmented_data[:len(data)]  # Para mantener el tamaño original


In [138]:
def agregar_wavelet(df,wavelet='db1'):
    data = df[df.columns[0]].values
    data_tf=wavelet_transform_augmentation(data,wavelet)
    df[df.columns[0]] = data_tf
    return df

DUPLICADO + PERTURBACIÓN:

In [150]:
def duplicate_and_perturb(data, duplication_factor=0.3, perturbation_std=0.05):
    duplicated_data = []
    for point in data:
        duplicated_data.append(point)
        if np.random.rand() < duplication_factor:
            duplicated_data.append(point + np.random.normal(0, perturbation_std))
    return np.array(duplicated_data)

In [152]:
def duplicados(df,freq,duplication_factor=0.3,perturbation_std=0.05):
    data = df[df.columns[0]]
    data_dd=duplicate_and_perturb(data,duplication_factor,perturbation_std)
    indice=series_periodos(df.index[0],len(data_dd),freq)
    df_dd=pd.DataFrame(data=data_dd,index=indice,columns=df.columns)
    return df_dd

SIMULACIÓN DE SALTOS:

In [156]:
def pulse_noise(data, num_pulses=5, amplitude=1):
    pulse_indices = np.random.choice(len(data), num_pulses, replace=False)
    pulse_data = data.copy()
    for i in pulse_indices:
        pulse_data[i] += np.random.uniform(-amplitude, amplitude)
    return pulse_data

In [157]:
def agregar_saltos(df,num_pulses=5,amplitude=1):
    data = df [df.columns[0]]
    data_salto=pulse_noise(data,num_pulses,amplitude)
    df[df.columns[0]]= data_salto
    return df

COMBINACIÓN LINEAL:

In [179]:
def linear_combinations(data, n_combinations):
    combinations = []
    for _ in range(n_combinations):
        weights = np.random.rand(data.shape[0])
        weights /= np.sum(weights)  # Normalizar pesos
        combination = np.dot(weights, data)
        combinations.append(combination)
    return np.array(combinations)

In [183]:
def agregar_comb(df,freq,size):
    data = df[df.columns[0]]
    data_augmented = linear_combinations(data,size)
    datos=np.concatenate((data.values,data_augmented))
    indice=series_periodos(df.index[0],len(datos),freq)
    df_dl=pd.DataFrame(data=datos,index=indice,columns=df.columns)
    return df_dl